<a href="https://colab.research.google.com/github/joaopaulof19/dados_pib_paises/blob/main/webScraping_ETL_project_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# Importando as bibliotecas necessárias
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import sqlite3
from datetime import datetime


In [68]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
db_name = "World_Economies.db"
table_name = "Countries_by_GDP"
csv_path = "Countries_by_GDP.csv"
table_attribs = ["Country", "GDP_USD_millions"]

In [70]:
# Código para operações ETL em dados do PIB do país

def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df

def transform(df):
    df["GDP_USD_millions"] = round(df["GDP_USD_millions"].str.replace(",", "").astype(float)/1000, 2)
    df.rename(columns={"GDP_USD_millions": "GDP_USD_billions"}, inplace=True)
    return df

def load_to_csv(df, csv_path):
    df.to_csv(csv_path, index=False)

def load_to_db(df, sql_connection, table_name):
    df.to_sql(name=table_name, con=sql_connection, if_exists="replace", index=False)

def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Ano-MêsNome-Dia-Hora-Minuto-Segundo
    now = datetime.now() # obter carimbo de data/hora atual
    timestamp = now.strftime(timestamp_format)
    print(timestamp + ' : ' + message + '\n')
    with open("./etl_project_log.txt","a") as f:
        f.write(timestamp + ' : ' + message + '\n')

In [66]:
log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('World_Economies.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

sql_connection.close()

2024-Jan-18-21:48:47 : Preliminaries complete. Initiating ETL process

2024-Jan-18-21:48:49 : Data extraction complete. Initiating Transformation process

2024-Jan-18-21:48:49 : Data transformation complete. Initiating loading process

2024-Jan-18-21:48:49 : Data saved to CSV file

2024-Jan-18-21:48:49 : SQL Connection initiated.

2024-Jan-18-21:48:49 : Data loaded to Database as table. Running the query

SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100
          Country  GDP_USD_billions
0   United States          26854.60
1           China          19373.59
2           Japan           4409.74
3         Germany           4308.85
4           India           3736.88
..            ...               ...
64          Kenya            118.13
65         Angola            117.88
66           Oman            104.90
67      Guatemala            102.31
68       Bulgaria            100.64

[69 rows x 2 columns]
2024-Jan-18-21:48:49 : Process Complete.

